## Load the dataset


In [1]:
from VQA_Dataset import VQA_Dataset
import clip
import torch
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model, preprocess = clip.load("ViT-B/32", device=device)

dataset = VQA_Dataset()
dataset.load_all(preprocess, device, length=200)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

c:\Users\mr_pa\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda device


Preprocessing Images: 100%|██████████| 200/200 [00:02<00:00, 87.12it/s]


## Simple model architectue


In [2]:
import tqdm

class VQA_Model(torch.nn.Module):
    """uses the model from clip like in evaluation method"""
    def __init__(self, model, device):
        super().__init__()
        self.model = model
        self.device = device

    def forward(self, image, question_tokens, answer_tokens):
        """returns the logits for the answers"""
        image_features = model.encode_image(image)
        question_features = model.encode_text(question_tokens)
        answer_features = model.encode_text(answer_tokens)
        
        answer_features /= answer_features.norm(dim=-1, keepdim=True)
        
        combined_features = image_features * question_features
        combined_features /= combined_features.norm(dim=-1, keepdim=True)

        similarity = (100.0 * combined_features @ answer_features.T).softmax(dim=-1)
        return similarity

    


In [26]:
class VQA_Model2(torch.nn.Module):
    """architecture that uses clip and a small NN to combine question and image features to get a embedding of the same size as the answer embedding
        Only train the NN, not the clip model
    """
    def __init__(self, model, device):
        super().__init__()
        self.model = model
        self.device = device
        self.fc1 = torch.nn.Linear(1024, 512).to(self.device).to(torch.float16)
        self.initialize_parameters()

    def initialize_parameters(self):
        # Apply Xavier/Glorot initialization to the linear layer
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.zeros_(self.fc1.bias.data)

    def forward(self, image, question_tokens, answer_tokens):

        """returns the logits for the answers"""
        image_features = model.encode_image(image)
        question_features = model.encode_text(question_tokens)
        answer_features = model.encode_text(answer_tokens)
        
        answer_features /= answer_features.norm(dim=-1, keepdim=True)
        
        # concatenate the features
        combined_features = torch.cat((image_features, question_features), dim=1).to(self.device)            
        combined_features /= combined_features.norm(dim=-1, keepdim=True)
        
        combined_features = self.fc1(combined_features)

        similarity = (100.0 * combined_features @ answer_features.T)# .softmax(dim=-1) no softmax because of cross entropy loss
        return similarity

## Evaluate

In [4]:
import tqdm

def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    for i, data in enumerate(tqdm.tqdm(dataloader)):
        image = data['image'].to(device)
        question_tokens = data['question_tokens'].to(device)
        answer_tokens = data['answer_tokens'].squeeze(0).to(device)
        image_id = int(data['image_id'])
        correct_answer = int(data['correct_answer'])

        with torch.no_grad():
            similarity = model(image, question_tokens, answer_tokens)
            pred = similarity.argmax(dim=-1).item()

            if pred == correct_answer:
                correct += 1
    print(f"Accuracy: {correct / len(dataloader)} correct: {correct} total: {len(dataloader)}")


In [5]:
combined_model = VQA_Model(model, device)
evaluate(combined_model, dataloader, device)

100%|██████████| 200/200 [00:10<00:00, 18.91it/s]

Accuracy: 0.04 correct: 8 total: 200


## Training

In [24]:
def train(model, dataloader, device, epochs=10):
    model.train()
    print(model.parameters())
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-7)
    loss_fn = torch.nn.CrossEntropyLoss()
    for epoch in range(epochs):
        for i, data in enumerate(tqdm.tqdm(dataloader)):
            image = data['image'].to(device)
            question_tokens = data['question_tokens'].to(device)
            answer_tokens = data['answer_tokens'].squeeze(0).to(device)
            image_id = int(data['image_id'])
            correct_answer = int(data['correct_answer'])

            optimizer.zero_grad()
            similarity = model(image, question_tokens, answer_tokens)

            # transform asnwer to tensor of the same shape as similarity before only correct index
            one_hot_encoding = torch.zeros(similarity.squeeze(0).shape[0])
            one_hot_encoding[correct_answer] = 1
            one_hot_encoding = one_hot_encoding.to(device)

            loss = loss_fn(similarity.squeeze(0), one_hot_encoding)
            loss.backward()
            optimizer.step()
            #print("--------------------------------------------------")
        print(f"Epoch {epoch} loss: {loss.item()}")
        evaluate(model, dataloader, device)

In [27]:
trained_model = VQA_Model2(model, device)
# freeze the clip model
for param in trained_model.model.parameters():
    param.requires_grad = False

train(trained_model, dataloader, device, epochs=2)

# evaluate the model
evaluate(trained_model, dataloader, device)

<generator object Module.parameters at 0x0000015AB3F4BED0>


100%|██████████| 200/200 [00:05<00:00, 39.40it/s]


Epoch 0 loss: nan


100%|██████████| 200/200 [00:05<00:00, 38.49it/s]


Accuracy: 0.0 correct: 0 total: 200


...

Epoch 1 loss: nan


100%|██████████| 200/200 [00:05<00:00, 35.54it/s]


Accuracy: 0.0 correct: 0 total: 200


100%|██████████| 200/200 [00:05<00:00, 36.40it/s]

Accuracy: 0.0 correct: 0 total: 200
